## Functions for fetching player height data
#### NOTE: Large queries will raise a flag server side. You probably want to limit to <500 IDs

In [9]:
import pandas as pd 
import requests
from typing import List

def parse_height(height_str: str) -> float:
    """
    Converts the height string from ft and in to ft

    Parameters:
    height_str: Height string in the format 'ft in'

    Returns:
    Height in ft (float)
    """
    # split str into ft and in
    feet, inches = height_str.split("' ")
    feet = int(feet)
    inches = int(inches.replace('"', ''))
    
    # convert the height to ft units
    height_in_feet = feet + inches / 12.0
    return height_in_feet

def get_player_heights(player_ids: List[int]) -> pd.DataFrame:
    """
    Fetches the heights of players given their IDs.
    
    Parameters:
    player_ids: List of player IDs.
    
    Returns:
    pandas DataFrame of player IDs, heights.
    """

    player_id_str = ','.join(map(str, player_ids))

    url = f'https://statsapi.mlb.com/api/v1/people?personIds={player_id_str}&fields=people,id,height,weight'

    # query the API endpoint
    response = requests.get(url)

    # raise an HTTPError if the request was unsuccessful
    response.raise_for_status()  
    
    json_data = response.json()['people'] # parse the JSON response
    data = pd.DataFrame(json_data) # convert to DataFrame
    data['height'] = data['height'].apply(parse_height) # convert height to ft
    
    return data

In [10]:
player_ids = [694973,573186]

player_df = get_player_heights(player_ids)

player_df.head()

,id,height,weight
0,694973,6.500000,235
1,573186,5.583333,180
